# Assignment 5
The script should not give an error. Any mistake will be consider as 0.


## Group 3 members:
- Alvaro Zapata: Create "for loop" code to select every election round.
- Diego Delgado: Create "for loop" code to select every election round.
- Guadalupe Ramirez: Create code for scrap tables.
- Daniela Flores: Create code for scrap tables.
- Luis Quispe: Join loops and create for loop to select tables if a pagination box exists.

In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [2]:
#!pip install selenium
#!pip install webdriver-manager
#!pip install pandas

In [3]:
#!pip3 install lxml
import lxml

In [4]:
#!pip install openpyxl
import openpyxl

In [5]:
# this library is to manipulate browser
from selenium import webdriver

# it allows you to work with differen versions of drivers
# We call ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import time 
import random
from selenium.webdriver.common.by import By
import pandas as pd # for table import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [6]:
pwd

'C:\\Users\\luisc\\Documents\\GitHub\\Diplomado_PUCP\\Lecture_7\\Assignment_5'

# Loop final


In [7]:
#Run packages

# Create a dataframe to collect the results (out of the for loop)
results = pd.DataFrame()

#################################
# loop starts here
#################################

# Open driver

service = Service(executable_path="../chromedriver-win64/chromedriver.exe")
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# Enter in the website
url = 'https://infogob.jne.gob.pe/Eleccion'
driver.get(url)
time.sleep(7)


# First loop, to select all election rounds
for periodo in range(26, 1, -1): #Iterated over a range of electoral rounds (from 26 to 1 in the reverse direction)

    # Open first selection box **(relevant in the for loop)**
    proceso = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div') # use selenium to find an element using XPATH
    proceso.click() #a function is activated by clicking
    time.sleep(random.randint(5,6)) #Pause the script for 5 seconds

    # Select "PROCESO PRESIDENCIAL"
    presidencial = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
    presidencial.click() 
    time.sleep(random.randint(5,6))

    # Open second box
    eleccion = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
    eleccion.click()
    time.sleep(random.randint(5,6))
    
    # select every round
    periodo_xpath = f'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{periodo}]'
    periodo_element = driver.find_element(By.XPATH, periodo_xpath)
    periodo_element.click()

    # get text
    elecciones_name = periodo_element.text #the visible text of the page is captured
    time.sleep(random.randint(5,6))

    # Click in search box
    buscar = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
    buscar.click()
    time.sleep(random.randint(5,6))

    # Click "Resultados"
    resultados = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
    resultados.click()
    time.sleep(random.randint(5,6))

    # Get table
    raw_table = driver.find_element(By.XPATH, "/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]/div[2]" )
    raw_table_html = raw_table.get_attribute( 'innerHTML' ) #This captures all the HTML contained within the table element

    # Organize the table
    tabla = pd.read_html(raw_table_html)[0]
    tabla_final = tabla[['ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]  #keep the most important columns
    tabla_final.insert(0,'Elecciones', elecciones_name)            #insert the name of round of election in the fisrt column

    # Append the table in the results dataframe **(relevant in the for loop)**
    results = pd.concat([results, tabla_final], ignore_index=True)
    
    #Second loop to check extra pages in the html source text
    html_source = driver.page_source #Uses the full HTML of the page as it is after all interactions
    #reset number of pages counter
    pages=0

    if '<a href="javascript:void(0)">5</a>' in html_source:
        pages=5
    elif '<a href="javascript:void(0)">4</a>' in html_source:
        pages=4
    elif '<a href="javascript:void(0)">3</a>' in html_source:
        pages=3
    elif '<a href="javascript:void(0)">2</a>' in html_source:
        pages=2
    else:
        pass #"pages" will remain zero if none of the expected fragments are found in the HTML source

    # Third loop to get the next pages if they exists
    if pages>1:
        a = range(3, pages+2, 1)
        for n in a:
            time.sleep(5)
            page_xpath = f'/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]/div[4]/div[2]/ul/li[{n}]/a'
            next_page = driver.find_element(By.XPATH, page_xpath)
            next_page.click()

            # Get table
            raw_table = driver.find_element(By.XPATH, "/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]/div[2]" )
            raw_table_html = raw_table.get_attribute( 'innerHTML' )

            # Organize the table
            tabla = pd.read_html(raw_table_html)[0]
            tabla_final = tabla[['ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]  #keep the most important columns
            tabla_final.insert(0,'Elecciones', elecciones_name)            #insert the name of round of election in the fisrt column

            # Append the table in the results dataframe **(relevant in the for loop)**
            results = pd.concat([results, tabla_final], ignore_index=True)

    else:
        pass #If there is only one page, nothing is done

    # We go back to search for the remaining tables
    back = driver.find_element(By.XPATH , '/html/body/div[1]/section/div[1]/a[2]')
    back.click()
    time.sleep(random.randint(2,3))


In [8]:
# Quit from the driver
driver.close()

#See results
results

,Elecciones,ORGANIZACIÓN POLÍTICA,TOTAL VOTOS
0,PRESIDENCIAL 1931,UNION REVOLUCIONARIA,152149
1,PRESIDENCIAL 1931,PARTIDO APRISTA PERUANO,106088
2,PRESIDENCIAL 1931,PARTIDO DESCENTRALISTA,21950
3,PRESIDENCIAL 1931,PARTIDO COALICION NACIONAL,19640
4,PRESIDENCIAL 1936,PARTIDO SOCIAL DEMOCRATA,77042
...,...,...,...
199,PRESIDENCIAL 2021,VOTOS NULOS,1123027
200,PRESIDENCIAL 2021 - 2DA VUELTA,PARTIDO POLITICO NACIONAL PERU LIBRE,8836380
201,PRESIDENCIAL 2021 - 2DA VUELTA,FUERZA POPULAR,8792117
202,PRESIDENCIAL 2021 - 2DA VUELTA,VOTOS EN BLANCO,121489


In [9]:
# After finishing the loop, export the result table as an Excel file

results.to_excel('Results_group3.xlsx', index=False)